# Preprocesamiento de datos

## 1. Introducción

El aprendizaje automático se ha convertido en una herramienta esencial para la clasificación, análisis y procesamiento de información en distintas áreas de la ciencia. En particular, la botánica enfrenta el reto de manejar grandes cantidades de imágenes de hojas de distintas especies, las cuales pueden presentar variaciones en iluminación, tamaño y calidad.

El presente documento describe el preprocesamiento de datos realizado para el proyecto de clasificación automática de hojas de Costa Rica. Este preprocesamiento es esencial para mejorar la calidad del dataset y preparar las imágenes para el entrenamiento de una red neuronal convolucional (CNN), asegurando que el modelo aprenda de manera eficiente y precisa, minimizando problemas de overfitting o underfitting.

## 2. Materiales y métodos

### Importar librerías

In [1]:
import os
import pandas as pd
import cv2
import shutil
import random
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import (
    ImageDataGenerator,
    array_to_img,
    img_to_array,
    load_img,
)

### Exploración inicial del dataset

In [ ]:
# Ruta del dataset original
dataset_path = "./All_CR_Leaves_Cleaned"

# Exploración inicial
all_classes = []
dir_elements = sorted(os.listdir(dataset_path))

for class_name in dir_elements:
    class_path = os.path.join(dataset_path, class_name)
    if os.path.isdir(class_path):
        images = [
            f
            for f in os.listdir(class_path)
            if f.lower().endswith((".png", ".jpg", ".jpeg"))
        ]
        all_classes.append([class_name, len(images)])

classes_DF = pd.DataFrame(
    all_classes, columns=["Class", "Number of Images"]
).sort_values(by="Number of Images", ascending=False)
print(f"Número de clases originales: {len(classes_DF)}")
classes_DF.head()

Número de clases originales: 255


,Class,Number of Images
242,Terminalia amazonia,89
40,Calycophyllum candidissimum,76
237,Tabebuia ochracea,67
233,Swietenia macrophylla,66
156,Manilkara chicle,65


### Visualización inicial

In [ ]:
plt.figure(figsize=(10, 50))
plt.barh(classes_DF["Class"], classes_DF["Number of Images"])
plt.title("Distribución inicial de imágenes por clase")
plt.xlabel("Número de imágenes")
plt.ylabel("Clase")
plt.show()

### Filtrado de clases con menos de 10 imágenes

In [12]:
filtered_classes_DF = classes_DF[classes_DF["Number of Images"] >= 10]
print(
    f"Clases después del filtrado (< 10 imágenes eliminadas): {len(filtered_classes_DF)}"
)
filtered_classes_DF.head()

filtered_classes_DF.to_csv("filtered_dataset_summary.csv", index=False)

Clases después del filtrado (< 10 imágenes eliminadas): 236


###  Reescalado de imágenes

In [20]:
input_dir = "./All_CR_Leaves_Cleaned"
output_dir = "./filtered_leaves"
img_size = (64, 64)

# Crear carpeta destino si no existe
os.makedirs(output_dir, exist_ok=True)

for class_name in os.listdir(input_dir):
    class_path = os.path.join(input_dir, class_name)
    if os.path.isdir(class_path):
        # Crear subcarpeta por clase en destino
        output_class_path = os.path.join(output_dir, class_name)
        os.makedirs(output_class_path, exist_ok=True)

        for img_name in os.listdir(class_path):
            if img_name.lower().endswith((".jpg", ".jpeg", ".png")):
                src_img_path = os.path.join(class_path, img_name)
                dst_img_path = os.path.join(output_class_path, img_name)

                img = cv2.imread(src_img_path)
                if img is None:
                    print(f"⚠ Imagen inválida: {src_img_path}")
                    continue

                resized_img = cv2.resize(img, img_size)
                cv2.imwrite(dst_img_path, resized_img)

print("✅ Reescalado y guardado en filtered_leaves")

✅ Reescalado y guardado en filtered_leaves


### Normalización brillo, contraste y píxeles

In [21]:
# Ruta donde estánlas imágenes ya reescaladas
processed_dir = "./filtered_leaves"


def normalize_image(img_path):
    # Leer imagen
    img = cv2.imread(img_path)
    if img is None:
        print(f"⚠ Imagen inválida: {img_path}")
        return

    #Convertir a escala de grises
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Aplicar ecualización de histograma en cada canal (mejor para imágenes a color)
    # Se separan los canales (B, G, R), se ecualiza cada uno y se recombinan.
    channels = cv2.split(img)
    eq_channels = []
    for ch in channels:
        eq_channels.append(cv2.equalizeHist(ch))
    img_eq = cv2.merge(eq_channels)

    # Normalizar píxeles al rango 0-1 dividiendo por 255 (mejora la estabilidad numérica en modelos)
    img_norm = img_eq / 255.0

    # Guardar de vuelta como imagen uint8 (rango 0-255), convirtiendo de nuevo de 0-1 a 0-255
    cv2.imwrite(img_path, (img_norm * 255).astype(np.uint8))


# Aplicar normalización a todas las imágenes dentro de ./filtered_leaves
for class_name in os.listdir(processed_dir):
    class_dir = os.path.join(processed_dir, class_name)
    if os.path.isdir(class_dir):
        for img_name in os.listdir(class_dir):
            if img_name.lower().endswith((".jpg", ".jpeg", ".png")):
                img_path = os.path.join(class_dir, img_name)
                normalize_image(img_path)

print("✅ Normalización de brillo, contraste y píxeles completada en filtered_leaves")

✅ Normalización de brillo, contraste y píxeles completada en filtered_leaves


## Discusión de resultados

## Avance y revisión del plan

## Conclusiones